In [ ]:
import os 
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../")))
from gitstats import NWBGitInfo, GitHubRepoInfo

In [ ]:
import git
import requests
from datetime import datetime
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

In [ ]:
git_repo_infos = NWBGitInfo.GIT_REPOS.get_info_objects()

In [ ]:
# Plot results for select repos
select_repos = ['PyNWB', 'HDMF', 'MatNWB', 'NWB_Schema', 'HDMF_Common_Schema']
fig, axes = plt.subplots(figsize=(16, len(select_repos)*4.2), 
                         nrows=len(select_repos), 
                         ncols=1, sharex=True, sharey=False, squeeze=True)
for i, repo in enumerate(select_repos):
    add_releases=None
    # Add 2.0 release for NWB schema. Same as PyNWB 1.0 release date
    if repo == "NWB_Schema":
        add_releases = [("2.0.0", datetime.strptime("2019-01-19", "%Y-%m-%d")),]
    ax = git_repo_infos[repo].plot_release_timeline(
        fontsize=14, 
        month_intervals=2,
        xlim=(datetime.strptime("2017-10-01", "%Y-%m-%d"), 
             datetime.strptime("2021-10-01", "%Y-%m-%d"),),
        ax=axes[i],
        title_on_yaxis=True,
        add_releases=add_releases)

axes[0].set_title("Timeline of NWB Release", fontdict={'fontsize':16})
plt.subplots_adjust(wspace=0, hspace=0.0)
plt.savefig('nwb_software_releases_timeline.pdf', dpi=300)
plt.show()

# Plot timeline with code stats

In [ ]:
from codestats import GitCodeStats
import pandas as pd
if GitCodeStats.cached('temp_cloc_stats'):
    git_code_stats = GitCodeStats.from_cache('temp_cloc_stats')
    date_range = pd.date_range(start=datetime.strptime(NWBGitInfo.HDMF_START_DATE, "%Y-%m-%d").strftime("%d %b %Y"), 
                               end=git_code_stats.cloc_stats['HDMF'][0]['date'], freq="D")
    code_summary_stats = git_code_stats.compute_summary_stats(date_range=date_range) 
    # Clean up HDMF stats to ignore data from before HDMF was extracted from PyNWB
    for k in code_summary_stats.keys():
        code_summary_stats[k]['HDMF'][:NWBGitInfo.HDMF_START_DATE] = 0
else:
    git_code_stats = None
    print("No cached code stats available")

In [ ]:
if git_code_stats:
    k = 'HDMF'
    curr_df = pd.DataFrame.from_dict({'code': code_summary_stats['codes'][k], 
                                      'blank': code_summary_stats['blanks'][k], 
                                      'comment': code_summary_stats['comments'][k]})
    curr_df.index = date_range
    ax = curr_df.plot.area(
        figsize=(18,10), 
        stacked=True, 
        linewidth=0, 
        fontsize=16)
    
    # Choose some nice levels
    names, dates = git_repo_infos[k].get_release_names_and_dates()
    version_jumps = GitHubRepoInfo.get_version_jump_from_tags(names)
    levels = [8000 if version_jumps[n] == "major" else 5000 if version_jumps[n] == "minor" else 2000 for n in names]
    
    ax.vlines(dates, 0, levels, color="black")  # The vertical stems.
    ax.plot(dates, np.zeros_like(dates), "-o",
            color="k", markerfacecolor="w")  # Baseline and markers on it.
    # annotate lines
    for d, l, r in zip(dates, levels, names):
        ax.annotate(r, xy=(d, l),
                    xytext=(14+2, np.sign(l)*3), textcoords="offset points",
                    horizontalalignment="right",
                    verticalalignment="bottom" if l > 0 else "top",
                    fontsize=20 if version_jumps[r] == "major" else 16 if version_jumps[r] == "minor" else 12,
                    color='black')

    plt.legend(loc=2, prop={'size': 16})
    plt.ylabel('Lines of Code (CLOC)', fontsize=16)
    plt.grid(color='black', linestyle='--', linewidth=0.7, axis='both')
    plt.title("Lines of Code: %s" % k, fontsize=20)
    plt.tight_layout()
    plt.show()
